In [1]:
from FDMBuilder.FDMTable import *
from FDMBuilder.FDMDataset import *
from FDMBuilder.testing_helpers import *

In [2]:
test_table_1 = FDMTable(
    source_table_id="CY_STAGING_DATABASE.src_BDCT_SRActivityEvent_010116_310822",
    dataset_id="CY_SAM_TEST"
)

In [3]:
test_table_1.copy_table_to_dataset()

In [11]:
def rename_columns_in_bigquery_test(table_id, names_map, verbose=True):
    """Renames columns of a table in bigquery

    Args:
        table_id: string, full id (project_id.dataset_id.table_id) of the table 
            containing the columns to be renamed
        names_map: dict, key-value pairs are strings, keys detailing
            current names, values the new names of the columns
        verbose: bool, True to output progress information to console, False to
            suppress all console output

    Returns:
        None - changes occurr in GCP

    Example:
    ```python
    # renames columns from old_name_1/2 to new_name_1/2
    rename_columns_in_bigquery(
        table_id = "project_id.dataset_id.table_id",
        names_map = {"old_name_1":"new_name_1, 
                     "old_name_2":"new_name_2}
    )
    ```
    """
    
    alias_list = []
    if verbose:
        print("\tRenaming Columns:")
    for old_name, new_name in names_map.items():
        alias_list.append(f"{old_name} AS {new_name}")
        if verbose:
            print(f"\t{old_name} -> {new_name}")
    alias_list.append(f"{list(names_map.keys())[0]} AS DUMMY_COL")
    alias_string = ", ".join(alias_list)
    old_names_string = ", ".join(names_map.keys())
    n_table_cols = len(get_table_schema_dict(table_id))
    
    if len(names_map) == n_table_cols:
        sql = f"""
            SELECT {alias_string}
            FROM `{table_id}`
        """
    else:
        sql = f"""
            SELECT {alias_string}, * EXCEPT({old_names_string})
            FROM `{table_id}`
        """

    run_sql_query(sql=sql, destination=table_id)
    drop_column_sql = f"""
        ALTER TABLE `{table_id}`
        DROP COLUMN DUMMY_COL
    """
    run_sql_query(drop_column_sql)
    if verbose:
        print("\tRenaming Complete\n")

In [12]:
rename_columns_in_bigquery_test(f"{PROJECT}.CY_SAM_TEST.src_BDCT_SRActivityEvent_010116_310822",
                                {"Person_id": "person_id"})

	Renaming Columns:
	Person_id -> person_id
	Renaming Complete



In [37]:
sql = """
SELECT DateEventRecorded AS dateeventrecorded, Person_id, * EXCEPT(DateEventRecorded, Person_id)
FROM yhcr-prd-phm-bia-core.CY_SAM_TEST.src_BDCT_SRActivityEvent_010116_310822
LIMIT 1000
"""
run_sql_query(sql, destination=f"{PROJECT}.CY_SAM_TEST.src_BDCT_SRActivityEvent_010116_310822")

Table(TableReference(DatasetReference('yhcr-prd-phm-bia-core', 'CY_SAM_TEST'), 'src_BDCT_SRActivityEvent_010116_310822'))

In [32]:
%%bigquery
SELECT DateEventRecorded, * EXCEPT(DateEventRecorded)
FROM yhcr-prd-phm-bia-core.CY_SAM_TEST.src_BDCT_SRActivityEvent_010116_310822
LIMIT 1000

Downloading: 100%|██████████| 1000/1000 [00:00<00:00, 1149.47rows/s]


,DateEventRecorded,rowidentifier,Person_id,DateEvent,IDProfileEnteredBy,IDBy,IDOrganisationAt,EventDetails,EventDetailsWith,EventDetailsMethod,EventDetailsDuration,IDEvent,IDReferralIn,IDPatient,IDOrganisation,IDOrganisationRegisteredAt
0,21 Mar 2019 15:17:25,937031587,13379057,21 Mar 2019 15:17:25,6957,6485,563468098101,-1,654,2781,45,50459485441,207918794,B86D44DD7A399576A87B8FF9D64A92BA6F52515A02DB33...,563468098101,563468098101
1,21 Nov 2018 12:00:57,876968907,13379057,21 Nov 2018 12:00:58,6957,6485,563468098101,-1,654,2781,60,48046982242,207918794,B86D44DD7A399576A87B8FF9D64A92BA6F52515A02DB33...,563468098101,563468098101
2,29 Nov 2018 10:08:24,887041015,13533193,29 Nov 2018 10:08:24,6957,6485,563468098101,-1,654,2781,45,47711178670,207885950,D513738D3F746CB54B0BC6E1F22D8302F4D8A077156D25...,563468098101,563468098101
3,17 May 2019 11:56:25,1044386141,12930157,17 May 2019 11:56:25,8015,6851,563468098101,-1,654,2781,30,49130715080,208083263,64CB42D6A2A10850D83E0C1051C5EFEE6A09C6298848BB...,563468098101,563468098101
4,21 Nov 2019 15:00:47,2201335730,12998540,21 Nov 2019 15:00:47,9427,4414,563468098101,-1,654,2781,30,19550067794,267280638,7183D4677CB1DEC52EA8C4B1CC9B7337840FD1D41F10B3...,563468098101,563468098101
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,11 Mar 2019 11:38:43,1015331196,13241765,11 Mar 2019 11:38:43,7242,4258,563468084105,628,654,2781,15,50339108404,227534817,9ED5CB1966F92EC68AEFCC8223F890AF5C8D97CF5259EE...,563468084105,563468084105
996,30 Nov 2018 23:55:27,974662834,13102605,30 Nov 2018 23:55:27,7242,4258,563468084105,628,654,2781,0,47617814119,207983657,85024CEC61241FF41A82F6F6C5E4CE25D0BEB81EBEE9CA...,563468084105,563468084105
997,15 Feb 2019 15:06:52,1006298024,13337567,15 Feb 2019 11:45:00,7242,4258,563468084105,628,654,2781,30,49664094092,206764660,B0B2B3851F729D423924524CF1D72EFE7956757C7F8C09...,563468084105,563468084105
998,16 Jul 2021 17:10:49,2735063621,13136633,16 Jul 2021 09:45:00,7242,4258,563468084105,628,654,2781,20,21776041506,680200640,8B533AE1A30537498F95E33E9D40CAACB090ECBC3313D7...,563468084105,563468084105
